Метод Зонтова

Конфигуарция

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import ast
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, Bidirectional, LSTM, TimeDistributed, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

Препроцесинг

In [2]:
def load_pressure_data_from_folder(folder_path):
    """
    Считывает все файлы из указанной папки folder_path,
    где каждый файл содержит 2 столбца (time, pressure) без заголовка.
    Возвращает словарь {file_id: DataFrame}.
    """
    data_dict = {}
    
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Берём имя файла без расширения в качестве идентификатора
        file_id = os.path.splitext(filename)[0]
        
        # Считываем файл без заголовка, предполагая разделитель - пробел
        try:
            df = pd.read_csv(
                file_path,
                header=None,               # нет строки с заголовками
                names=['time', 'pressure'],# задаём имена столбцов
                delim_whitespace=True      # столбцы разделены пробелами/табами
            )
            data_dict[file_id] = df
        except Exception as e:
            pass
    
    return data_dict

In [3]:
def load_ground_truth(ground_truth_path):
    """
    Считывает CSV с колонками [file, recovery, drop].
    Возвращает DataFrame, где recovery и drop – списки интервалов.
    """
    gt_df = pd.read_csv(ground_truth_path)
    for col in ['recovery', 'drop']:
        gt_df[col] = gt_df[col].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
    return gt_df

In [4]:
def fill_missing_values(df):
    """
    Заполняет пропущенные значения (NaN) в столбце 'pressure'
    с помощью линейной интерполяции.
    """
    df['pressure'] = df['pressure'].interpolate(method='linear')
    return df

In [5]:
def smooth_data(df, window_length=11, polyorder=2):
    """
    Применяет Savitzky–Golay фильтр для сглаживания данных.
    Параметры:
      - window_length: длина окна (нечётное число).
      - polyorder: порядок полинома для аппроксимации.
    """
    if len(df['pressure']) < window_length:
        # Если данных недостаточно, оставляем без сглаживания
        df['pressure_smoothed'] = df['pressure']
    else:
        df['pressure_smoothed'] = savgol_filter(df['pressure'], window_length, polyorder)
    return df

In [6]:
def normalize_data(df):
    """
    Нормализует сглаженные данные, приводя их значения к диапазону [0, 1].
    """
    min_val = df['pressure_smoothed'].min()
    max_val = df['pressure_smoothed'].max()
    df['pressure_normalized'] = (df['pressure_smoothed'] - min_val) / (max_val - min_val) if max_val != min_val else 0
    return df

In [7]:
def preprocess_df(df):
    df = fill_missing_values(df)
    df = smooth_data(df)
    df = normalize_data(df)
    return df

In [8]:
def label_time_series(df, gt_recovery, gt_drop):
    """
    Для каждого временного шага задаёт метку:
      0 – фон,
      1 – recovery (КВД),
      2 – drop (КПД).
    gt_recovery и gt_drop – списки интервалов, например [[start, end], ...].
    """
    labels = np.zeros(len(df), dtype=np.int32)
    times = df['time'].values
    for interval in gt_recovery:
        start, end = interval
        mask = (times >= start) & (times <= end)
        labels[mask] = 1
    for interval in gt_drop:
        start, end = interval
        mask = (times >= start) & (times <= end)
        labels[mask] = 2
    return labels

In [9]:
def create_windows(df, labels, window_size, step_size):
    """
    Разбивает временной ряд (df['pressure_normalized']) и соответствующие метки (labels)
    на окна длины window_size с шагом step_size.
    
    Возвращает:
      X_windows (np.array) – форма (N, window_size, 1),
      y_windows (np.array) – форма (N, window_size).
    Если не получилось сформировать ни одного окна или формы несовместимы, возвращаются пустые массивы.
    """
    X_windows = []
    y_windows = []
    
    # Берём нормализованный сигнал
    series = df['pressure_normalized'].values
    n_points = len(series)
    
    # Формируем окна
    for i in range(0, n_points - window_size + 1, step_size):
        X_windows.append(series[i:i + window_size])    # форма (window_size,)
        y_windows.append(labels[i:i + window_size])    # форма (window_size,)
    
    # Превращаем списки в numpy-массивы
    X_windows = np.array(X_windows)   # ожидаем (N, window_size)
    y_windows = np.array(y_windows)   # ожидаем (N, window_size)
    
    # Если нет ни одного окна, сразу возвращаем пустые массивы
    if X_windows.size == 0:
        return X_windows, y_windows  # оба (0,)
    
    # --- Исправляем форму X_windows ---
    if X_windows.ndim == 1:
        # Значит у нас ровно одно окно, и форма (window_size,)
        if X_windows.shape[0] == window_size:
            # Превращаем в (1, window_size)
            X_windows = X_windows[np.newaxis, ...]
        else:
            # Что-то пошло не так: пропорции не совпадают
            return np.array([]), np.array([])
    
    # Теперь, если X_windows.ndim == 2, это (N, window_size)
    if X_windows.ndim == 2:
        X_windows = np.expand_dims(X_windows, axis=-1)  # => (N, window_size, 1)
    
    # --- Исправляем форму y_windows ---
    # Для меток достаточно формы (N, window_size). Если одно окно, может получиться (window_size,)
    if y_windows.ndim == 1:
        # Значит ровно одно окно, (window_size,)
        if y_windows.shape[0] == window_size:
            y_windows = y_windows[np.newaxis, ...]      # => (1, window_size)
        else:
            return np.array([]), np.array([])
    
    # Проверяем, совпадает ли количество окон
    if X_windows.shape[0] != y_windows.shape[0]:
        return np.array([]), np.array([])
    

    
    return X_windows, y_windows



In [10]:
def build_model(time_steps, n_features, n_classes):
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu', padding='same', input_shape=(time_steps, n_features)))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
    optimizer = Adam(learning_rate=1e-3)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [11]:
def main():
    folder_path = "../data/raw/Task 21"           # Папка с файлами (time, pressure)
    ground_truth_path = "../data/raw/ground_truth.csv"  # Файл с ground_truth
    
    # Считываем данные
    data_dict = load_pressure_data_from_folder(folder_path)
    gt_df = load_ground_truth(ground_truth_path)
    
    window_size = 200  # Длина окна по временным шагам
    step_size = 50     # Шаг скользящего окна
    X_all = []
    y_all = []
    
    # Для каждого файла: предобработка, разметка и формирование окон
    for file_id, df in data_dict.items():
        df = preprocess_df(df)
        gt_row = gt_df[gt_df['file'] == file_id]
        if not gt_row.empty:
            gt_recovery = gt_row.iloc[0]['recovery']
            gt_drop = gt_row.iloc[0]['drop']
        else:
            gt_recovery, gt_drop = [], []
        labels = label_time_series(df, gt_recovery, gt_drop)
        X_windows, y_windows = create_windows(df, labels, window_size, step_size)
        if X_windows.size == 0:
            continue
        X_all.append(X_windows)
        y_all.append(y_windows)
    
    X_all = np.concatenate(X_all, axis=0)
    y_all = np.concatenate(y_all, axis=0)
    
    # Преобразуем метки в one-hot вектор для каждого временного шага
    n_classes = 3
    y_all_onehot = tf.keras.utils.to_categorical(y_all, num_classes=n_classes)
    
    print("Форма входных данных X:", X_all.shape)     # (n_windows, window_size, 1)
    print("Форма меток y:", y_all_onehot.shape)        # (n_windows, window_size, 3)
    
    # Разделяем данные на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X_all, y_all_onehot, test_size=0.2, random_state=42)
    
    # Строим модель
    time_steps = window_size
    n_features = 1
    model = build_model(time_steps, n_features, n_classes)
    
    # Обучаем модель
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)
    
    # Сохраняем модель в файл
    model_save_path = "model.h5"
    model.save(model_save_path)
    print(f"Модель сохранена в {model_save_path}")
    
    return model, history

In [12]:
main()

C:\Users\user\AppData\Local\Temp\ipykernel_11372\2779453277.py:17: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\user\AppData\Local\Temp\ipykernel_11372\2779453277.py:17: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\user\AppData\Local\Temp\ipykernel_11372\2779453277.py:17: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\user\AppData\Local\Temp\ipykernel_11372\2779453277.py:17: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\user\AppData\Local\Temp\ipykernel_11372\2779453277.py:17: FutureWarning: Th

KeyboardInterrupt: 